In [105]:
import numpy as np
import tensorly as tl
import matplotlib.pyplot as plt
import tensortools as tt
from tensortools.operations import unfold as tt_unfold, khatri_rao
from tensorly.decomposition import parafac

In [80]:
def decompose_three_way(tensor, rank, max_iter=501, verbose=False):

    # a = np.random.random((rank, tensor.shape[0]))
    b = np.random.random((rank, tensor.shape[1]))
    c = np.random.random((rank, tensor.shape[2]))

    for epoch in range(max_iter):
        # optimize a
        input_a = khatri_rao([b.T, c.T])
        target_a = tl.unfold(tensor, mode=0).T
        a = np.linalg.solve(input_a.T.dot(input_a), input_a.T.dot(target_a))

        # optimize b
        input_b = khatri_rao([a.T, c.T])
        target_b = tl.unfold(tensor, mode=1).T
        b = np.linalg.solve(input_b.T.dot(input_b), input_b.T.dot(target_b))

        # optimize c
        input_c = khatri_rao([a.T, b.T])
        target_c = tl.unfold(tensor, mode=2).T
        c = np.linalg.solve(input_c.T.dot(input_c), input_c.T.dot(target_c))

        if verbose and epoch % int(max_iter * .2) == 0:
            res_a = np.square(input_a.dot(a) - target_a)
            res_b = np.square(input_b.dot(b) - target_b)
            res_c = np.square(input_c.dot(c) - target_c)
            print("Epoch:", epoch, "| Loss (C):", res_a.mean(), "| Loss (B):", res_b.mean(), "| Loss (C):", res_c.mean())

    return a.T, b.T, c.T

In [155]:
def decompose_three_wayOrth(tensor, rank, max_iter=501, verbose=False):

    # a = np.random.random((rank, tensor.shape[0]))
    b = orth(np.random.random((rank, tensor.shape[1])).T).T
    c = np.random.random((rank, tensor.shape[2]))

    for epoch in range(max_iter):
        # optimize a
        input_a = khatri_rao([b.T, c.T])
        target_a = tl.unfold(tensor, mode=0).T
        a = np.linalg.solve(input_a.T.dot(input_a), input_a.T.dot(target_a))
        aT, _ = np.linalg.qr(a.T)
        a = aT.T
        #a = orth(a.T).T

        # optimize b
        input_b = khatri_rao([a.T, c.T])
        target_b = tl.unfold(tensor, mode=1).T
        b = np.linalg.solve(input_b.T.dot(input_b), input_b.T.dot(target_b))
        bT, _ = np.linalg.qr(b.T)
        b = bT.T
        #b = orth(b.T).T

        # optimize c
        input_c = khatri_rao([a.T, b.T])
        target_c = tl.unfold(tensor, mode=2).T
        c = np.linalg.solve(input_c.T.dot(input_c), input_c.T.dot(target_c))
        #c = orth(c.T).T

        if verbose and epoch % int(max_iter * .05) == 0:
            res_a = np.square(input_a.dot(a) - target_a)
            res_b = np.square(input_b.dot(b) - target_b)
            res_c = np.square(input_c.dot(c) - target_c)
            print("Epoch:", epoch, "| Loss (C):", res_a.mean(), "| Loss (B):", res_b.mean(), "| Loss (C):", res_c.mean())

    return a.T, b.T, c.T

In [140]:
mat1 = np.random.randn(3, 4)
mat2 = np.random.randn(3, 4)
mat3 = np.random.randn(3, 4)
tsor = np.array([mat1, mat2, mat3])

In [168]:
res = decompose_three_way(tsor, 2, 10000, True)

Epoch: 0 | Loss (C): 0.8373826341242735 | Loss (B): 0.7952302274921632 | Loss (C): 0.6309718300068216
Epoch: 2000 | Loss (C): 0.3370596972073261 | Loss (B): 0.33705969720732615 | Loss (C): 0.33705969720732604
Epoch: 4000 | Loss (C): 0.33705969720732604 | Loss (B): 0.33705969720732615 | Loss (C): 0.33705969720732604
Epoch: 6000 | Loss (C): 0.3370596972073261 | Loss (B): 0.3370596972073261 | Loss (C): 0.33705969720732604
Epoch: 8000 | Loss (C): 0.33705969720732604 | Loss (B): 0.3370596972073261 | Loss (C): 0.33705969720732604


In [171]:
resOrth1 = decompose_three_wayOrth(tsor, 2, 10000, True)

Epoch: 0 | Loss (C): 1.0230546818999706 | Loss (B): 0.8790713188652618 | Loss (C): 0.5410306231533807
Epoch: 500 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 1000 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 1500 | Loss (C): 0.42230948596268125 | Loss (B): 0.42230948596268136 | Loss (C): 0.4223094859626813
Epoch: 2000 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 2500 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 3000 | Loss (C): 0.42230948596268125 | Loss (B): 0.42230948596268136 | Loss (C): 0.4223094859626813
Epoch: 3500 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 4000 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 4500 | Loss (C): 0.42230948596268125 | Loss (B):

In [172]:
resOrth2 = decompose_three_wayOrth(tsor, 2, 10000, True)

Epoch: 0 | Loss (C): 0.9686015212828428 | Loss (B): 0.9507544606275133 | Loss (C): 0.61641940204718
Epoch: 500 | Loss (C): 0.42230948596268125 | Loss (B): 0.42230948596268136 | Loss (C): 0.4223094859626813
Epoch: 1000 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 1500 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 2000 | Loss (C): 0.42230948596268125 | Loss (B): 0.42230948596268136 | Loss (C): 0.4223094859626813
Epoch: 2500 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 3000 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 3500 | Loss (C): 0.42230948596268125 | Loss (B): 0.42230948596268136 | Loss (C): 0.4223094859626813
Epoch: 4000 | Loss (C): 0.4223094859626813 | Loss (B): 0.4223094859626813 | Loss (C): 0.4223094859626813
Epoch: 4500 | Loss (C): 0.4223094859626813 | Loss (B): 

In [173]:
resOrth1[0] - resOrth2[0]

array([[ 0.00000000e+00,  5.55111512e-17],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.11022302e-16]])

In [111]:
res = decompose_three_way(tsor, 2, 10000, True)
res1 = parafac(tsor, 2)
res2 = parafac(tsor, 2)

Epoch: 0 | Loss (C): 0.9713199811007157 | Loss (B): 0.9187808075956014 | Loss (C): 0.7976875134356612
Epoch: 2000 | Loss (C): 0.42404130100886334 | Loss (B): 0.4240413010088633 | Loss (C): 0.4240413010088633
Epoch: 4000 | Loss (C): 0.42404130100886334 | Loss (B): 0.4240413010088633 | Loss (C): 0.4240413010088633
Epoch: 6000 | Loss (C): 0.42404130100886334 | Loss (B): 0.4240413010088633 | Loss (C): 0.4240413010088633
Epoch: 8000 | Loss (C): 0.42404130100886334 | Loss (B): 0.4240413010088633 | Loss (C): 0.4240413010088633


In [112]:
res1.factors = res

In [123]:
np.sum(orth(res[0])[:, 0] * orth(res[0])[:, 1])

4.3368086899420177e-16